# Imports

In [1]:
import nfl_data_py as nfl
import pandas as pd

# Loading Depth Chart Data

In [10]:
df_depth_chart = nfl.import_depth_charts(range(2001, 2023 + 1))

In [11]:
df_depth_chart.columns

Index(['season', 'club_code', 'week', 'game_type', 'depth_team', 'last_name',
       'first_name', 'football_name', 'formation', 'gsis_id', 'jersey_number',
       'position', 'elias_id', 'depth_position', 'full_name'],
      dtype='object')

In [20]:
df_depth_chart.depth_team.unique()

array(['1', '2', '3'], dtype=object)

In [23]:
non_numeric_values = df_depth_chart[~df_depth_chart['depth_team'].str.isnumeric()]['depth_team'].unique()
print(non_numeric_values)

[]


In [24]:
df_depth_chart['depth_team'] = pd.to_numeric(df_depth_chart['depth_team'], errors='coerce')

In [39]:
df_depth_chart_seasonal = df_depth_chart \
    .groupby(by=["season", 'gsis_id']) \
    .agg({"depth_team": ["mean", "min"]})

# Flatten the multi-level columns
df_depth_chart_seasonal.columns = ['_'.join(col).strip() for col in df_depth_chart_seasonal.columns.values]

# Reset the index if you want to convert the grouped indices to columns
df_depth_chart_seasonal.reset_index(inplace=True)

df_depth_chart_seasonal.rename(columns={"gsis_id": "player_id"}, inplace=True)

In [40]:
df_depth_chart_seasonal.head()

,season,player_id,depth_team_mean,depth_team_min
0,2001.0,00-0000007,2.000000,2
1,2001.0,00-0000017,1.608696,1
2,2001.0,00-0000032,2.000000,2
3,2001.0,00-0000045,1.000000,1
4,2001.0,00-0000065,1.000000,1


# Loading Core Data

In [36]:
df_core = pd.read_pickle("../../FantasyData/data-frames/df_basic_ngs_snaps_adv_redzone-tgts_team-snap-data_1999_2023.pkl")

In [37]:
df_core.head()

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,ybc_att_pfr_rush,yac_pfr_rush,yac_att_pfr_rush,brk_tkl_pfr_rush,att_br_pfr_rush,loaded_pfr_rush,redzone_targets,Att_passing_totals,Att_rushing_totals,total_snaps
0,00-0000003,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
1,00-0000003,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
2,00-0000003,2000,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,00-0000007,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,00-0000007,2000,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


# Merging

In [41]:
df_merge = df_core.merge(df_depth_chart_seasonal, on=["season", "player_id"], how="left")

In [42]:
df_merge.head()

,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,yac_att_pfr_rush,brk_tkl_pfr_rush,att_br_pfr_rush,loaded_pfr_rush,redzone_targets,Att_passing_totals,Att_rushing_totals,total_snaps,depth_team_mean,depth_team_min
0,00-0000003,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
1,00-0000003,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
2,00-0000003,2000,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,00-0000007,1999,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,00-0000007,2000,REG,0,0,0.0,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


# Save Data Frame

In [47]:
df_merge.to_pickle("../../FantasyData/data-frames/df_basic_ngs_snaps_adv_redzone-tgts_team-snap-data_depth-chart_1999_2023.pkl")